In [ ]:
# Import

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import scipy as sp

import math
import torch
import pygsp
import optuna
import joblib
import gc
import argparse
import os
import matplotlib
import pickle
import glob

from matplotlib.ticker import ScalarFormatter, StrMethodFormatter, FormatStrFormatter, FuncFormatter
from matplotlib.animation import FuncAnimation
from matplotlib.colors import ListedColormap

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, confusion_matrix, auc, f1_score
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm
from optuna.samplers import TPESampler, BruteForceSampler

from torch import nn
from torch.nn import Linear, Conv1d, LayerNorm, DataParallel
from torch_geometric.nn import GCNConv, Sequential, GraphConv
from torch_geometric.nn.dense import mincut_pool, dense_mincut_pool
from torch_geometric.utils import dense_to_sparse
from torch.nn.functional import glu

from torch_geometric.nn.models import GraphUNet
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx, grid

import sensors.utils.fault_detection as fd
import sensors.utils.analysis as ana
import sensors.utils.utils as utils

from importlib import reload
ana = reload(ana)
utils = reload(utils)


from pyprojroot import here
ROOT_DIR = str(here())
data_dir = '~/data/interim/'

matplotlib.rcParams.update({'font.size': 20})
matplotlib.rcParams.update({'font.family': 'DejaVu Serif'})

In [ ]:
dataset = 'df_StOlavs_D1L2B'
df_orig = pd.read_parquet(data_dir + f'{dataset}.parq')

np.random.seed(0)

df, nodes = fd.treat_nodes(df_orig)
G, nodes['subgraph'] = fd.NNGraph(nodes, radius=15, subgraphs=True)

main_graph = nodes.subgraph.value_counts().index[0]
nodes = nodes.query('subgraph==@main_graph')
G = fd.NNGraph(nodes, radius=15)
df = df[df.pid.isin(nodes.pid.unique())]

nodes['cluster'] = KMeans(n_clusters=20, n_init='auto').fit_predict(nodes[['northing','easting']])
df.drop('cluster', axis=1, inplace=True)
df = df.merge(nodes[['pid','cluster']], how='left', on='pid')
df['strcluster'] = df.cluster.astype(str).values

df_anomaly = df.copy()
df_anomaly['anomaly'] = 0

# # Anomaly test - different anomalous clusters
# # Anomaly 1
# anomaly_sensor = (df_anomaly.cluster==0)
# anomaly_onset = (df_anomaly.timestamp>'Jul 2020')&(df_anomaly.timestamp<'Jan 2021')
# anomaly_loc = anomaly_sensor&anomaly_onset

# df_anomaly.loc[anomaly_loc, 'smoothed'] += 0
# df_anomaly.loc[anomaly_loc, 'anomaly'] = 1

# Anomaly 2
anomaly_sensor = (df_anomaly.cluster==12)
anomaly_onset = (df_anomaly.timestamp>'Jul 2020')
anomaly_loc = anomaly_sensor&anomaly_onset

df_anomaly.loc[anomaly_loc, 'smoothed'] += 5
df_anomaly.loc[anomaly_loc, 'anomaly'] = 2

# Anomaly 3
anomaly_sensor = (df_anomaly.cluster==19)
anomaly_onset = (df_anomaly.timestamp>'Jul 2020')
anomaly_loc = anomaly_sensor&anomaly_onset

df_anomaly.loc[anomaly_loc, 'smoothed'] -= 5
df_anomaly.loc[anomaly_loc, 'anomaly'] = 3

X = df_anomaly.pivot(index='pid', columns='timestamp', values='smoothed').values
X = torch.tensor(X)

label = df_anomaly.pivot(index='pid', columns='timestamp', values='anomaly').values.max(axis=1)

fig, ax = plt.subplots(figsize=(8,5))
G.plot_signal(label, ax=ax, plot_name='')

# label_cmap = ListedColormap(['blue','goldenrod', 'sienna','aquamarine'])
label_cmap = ListedColormap(plt.cm.viridis(np.linspace(0,1,df_anomaly.anomaly.nunique())))

ax.collections[0].set_cmap(label_cmap)  # Modify the colormap of the plotted data
ax.axis('off')
plt.show()

In [ ]:
# Number of clusters for each feature
def kmeans_features(data, num_clusters):

    def cluster_kmeans(tensor, k):
        kmeans = KMeans(n_clusters=k, n_init=1)
        kmeans.fit(tensor)
        return kmeans.labels_

    kmeans_features = []
    # Perform clustering for each number of clusters
    for k in num_clusters:
        # Perform K-means clustering
        cluster_labels = cluster_kmeans(data, k)
        kmeans_features.append(cluster_labels)

    return torch.tensor(np.array(kmeans_features).T)

In [ ]:
class ClusterTS(nn.Module):
    def __init__(self,
                 conv1d_n_feats, conv1d_kernel_size, conv1d_stride,
                 graphconv_n_feats,
                 n_timestamps,
                 n_clusters,
                 n_extra_feats):
        
        super(ClusterTS, self).__init__()
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=conv1d_n_feats,
                                kernel_size=conv1d_kernel_size, stride=conv1d_stride)
        
        self.L_in = n_timestamps
        self.L_out = math.floor((self.L_in - conv1d_kernel_size)/conv1d_stride + 1)

        self.conv1d_out = conv1d_n_feats*self.L_out
        
        mlp_in = self.conv1d_out + n_extra_feats
        self.mcp_mlp = Linear(mlp_in, n_clusters)
    
    def forward(self, X, A, extra_feats=None):

        #HP
        factor_coords = 0.5

        # Data
        X = X.float()
        norm_X = LayerNorm(X.shape, elementwise_affine=False)
        X = norm_X(X)

        X = X.unsqueeze(1) # adjusting shape for conv1d
        X = self.conv1d(X)

        X = X.reshape((X.shape[0],-1)) #

        if extra_feats is not None:
            norm_f = LayerNorm(extra_feats.shape, elementwise_affine=False)
            extra_feats = factor_coords*norm_f(extra_feats)
            X = torch.cat((X,extra_feats),dim=1)

        S = self.mcp_mlp(X)

        _, _, loss_mc, loss_o = dense_mincut_pool(X, A, S)

        # return torch.softmax(S, dim=-1), loss_mc, loss_o
        return S, loss_mc, loss_o, X, C
    

# torch.random.manual_seed(0)

conv1d_n_feats = 3
conv1d_kernel_size = 60
conv1d_stride = 30

graphconv_n_feats = 30

n_nodes = X.shape[0]
n_timestamps = X.shape[1]

n_clusters = 20
factor = 0.25

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = X.to(device)

# Node coordinates
C = torch.tensor(G.coords)
A = torch.tensor(G.W.toarray()).float() #Using W as a float() tensor
A = A.to(device)

num_clusters_per_feature = [5, 6]
kmeans_feats = kmeans_features(C, num_clusters_per_feature).to(device).float()
n_extra_feats = kmeans_feats.shape[1]

# kmeans_feats = None
# n_extra_feats = 0

model = ClusterTS(conv1d_n_feats, conv1d_kernel_size, conv1d_stride, graphconv_n_feats,
                  n_timestamps, n_clusters, n_extra_feats)
# model.conv1d = DataParallel(model.conv1d)
# model.mcp_mlp = DataParallel(model.mcp_mlp)
model = model.to(device)

print(f'Clusters: {n_clusters}')
print(f'Factor: {factor}')

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def train(N_epochs):
 
    loss_evo = []
    loss_mc_evo = []
    loss_o_evo = []

    model.train()
    for epoch in tqdm(range(N_epochs)):

        optimizer.zero_grad()
        _, loss_mc, loss_o, Xm, Cm = model(X, A, kmeans_feats)
        loss = loss_mc + factor*loss_o
        loss.backward()
        optimizer.step()
        loss_evo.append(loss.item())
        loss_mc_evo.append(loss_mc.item())
        loss_o_evo.append(loss_o.item())

    return loss_evo, loss_mc_evo, loss_o_evo


loss_evo, loss_mc_evo, loss_o_evo = train(10000)

model.eval()
S, _, _, Xm, Cm = model(X, A, kmeans_feats)
S = S.cpu()


print(model)

x_values = range(1, len(loss_evo) + 1)
df_loss = pd.DataFrame({'Iteration': x_values, 'Loss': loss_evo, 'L_mc': loss_mc_evo, 'L_o': loss_o_evo})

# Melt the DataFrame to have a single column for Loss Type
df_loss = df_loss.melt(['Iteration'], ['Loss', 'L_mc', 'L_o'], 'Loss Type', 'Loss Value')

# Plot the lines using Plotly Express
fig = px.line(df_loss, x='Iteration', y='Loss Value', color='Loss Type',width=700).show()


fig, ax = plt.subplots(ncols=2, figsize=(16,5))
plotting_params = {'edge_color':'darkgray', 'edge_width':1.5,'vertex_color':'black', 'vertex_size':50}
G.plotting.update(plotting_params)
G.plot_signal(label, ax=ax[0], plot_name='')

ax[0].collections[0].set_cmap(label_cmap)  # Modify the colormap of the plotted data
ax[0].axis('off')

print(S.argmax(dim=1).unique())
G.plot_signal(np.array(S.argmax(dim=1)), ax=ax[1], plot_name='')
ax[1].collections[0].set_cmap('viridis')
ax[1].axis('off')

plt.show()

fig, ax = plt.subplots(figsize=(7,5))
G.plot_signal(S.softmax(dim=-1).detach().numpy().max(axis=1), ax=ax, plot_name='')
ax.collections[0].set_cmap('viridis')
ax.axis('off')
plt.show()

In [ ]:
nodes['pred'] = np.array(S.argmax(dim=1))
nodes['score'] = S.softmax(dim=-1).detach().numpy().max(axis=1)
nodes['anomaly'] = df_anomaly[['pid','anomaly']].groupby('pid').anomaly.max().values

utils.visualize_map(nodes, color='pred', size=np.ones(nodes.pid.nunique()), size_max=10,
                     hover_data=['cluster'], zoom=15, figsize=(700,700), colormap='viridis')
utils.visualize_map(nodes, color='score', size=np.ones(nodes.pid.nunique()), size_max=10,
                     hover_data=['cluster'], zoom=15, figsize=(700,700), colormap='viridis')

In [ ]:
most_common_preds = nodes.query('anomaly!=0').groupby('anomaly')['pred'].apply(lambda x: x.mode()[0])
most_common_preds

nodes['new_pred'] = nodes['pred']
nodes.loc[~nodes.pred.isin(most_common_preds.values),'new_pred'] = 0

max_anomaly = nodes.groupby('new_pred')['anomaly'].transform('max')

# Replace non-zero values of 'new_pred' with the maximum 'anomaly' for that 'new_pred'
nodes.loc[nodes['new_pred'] != 0, 'new_pred'] = max_anomaly

utils.visualize_map(nodes, color='new_pred', size=np.ones(nodes.pid.nunique()), size_max=10,
                     hover_data=['anomaly'], zoom=15, figsize=(700,700), colormap='viridis')


In [ ]:
f1_score(y_true=nodes.anomaly, y_pred=nodes.new_pred, average='weighted')

In [ ]:
nodes.query('cluster==12').count()

In [ ]:
nodes.query('cluster==0').pred.value_counts().index[0]

In [ ]:
df_test = df_anomaly[['pid','cluster','anomaly']].copy().groupby('pid', as_index=False).max()
df_test.query('cluster==0')

In [ ]:
G.plot_signal(kmeans_feats.detach().numpy()[:,1])

In [ ]:
Xm.std(dim=0)

In [ ]:
# OLD

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class TimeSeriesNet(nn.Module):
#     def __init__(self, num_time_series, num_channels, kernel_size, hidden_dim):
#         super(TimeSeriesNet, self).__init__()
#         self.shared_conv_layer = nn.Conv1d(in_channels=num_channels, out_channels=hidden_dim, kernel_size=kernel_size)
#         self.graph_conv = GraphConvolution(input_dim=hidden_dim * num_time_series, output_dim=hidden_dim)

#     def forward(self, time_series_data):
#         # Concatenate all time series along the batch dimension
#         batched_time_series_data = torch.stack(time_series_data, dim=0)
        
#         # Apply shared 1D convolution along the time dimension for all time series
#         conv_out = self.shared_conv_layer(batched_time_series_data)
#         conv_out = F.relu(conv_out)  # Apply ReLU activation
        
#         # Reshape the convolutional output to have time series as channels
#         conv_out = conv_out.permute(0, 2, 1)  # Swap time and channel dimensions
#         conv_out = conv_out.view(conv_out.size(0), -1, conv_out.size(2))  # Reshape to (batch_size, num_time_series, sequence_length)
        
#         # Apply graph convolution along the channel dimension
#         graph_conv_out = self.graph_conv(conv_out)
#         return graph_conv_out

# # Example usage
# num_time_series = 3
# num_channels = 1
# kernel_size = 3
# hidden_dim = 64
# graph_conv_input_dim = hidden_dim * num_time_series  # Adjust based on the size of merged features
# graph_conv_output_dim = 64  # Adjust based on your task
# graph_conv = GraphConvolution(input_dim=graph_conv_input_dim, output_dim=graph_conv_output_dim)

# # Initialize your time series data
# time_series_data = [torch.randn(batch_size, num_channels, sequence_length) for _ in range(num_time_series)]

# # Create and forward through the network
# model = TimeSeriesNet(num_time_series=num_time_series, num_channels=num_channels, kernel_size=kernel_size, hidden_dim=hidden_dim)
# output = model(time_series_data)


In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class ClusterTS(nn.Module):
#     def __init__(self, num_time_series, num_channels, kernel_size, hidden_dim):
#         super(ClusterTS, self).__init__()
#         self.conv_layers = nn.ModuleList([
#             nn.Conv1d(in_channels=num_channels, out_channels=hidden_dim, kernel_size=kernel_size)
#             for _ in range(num_time_series)
#         ])
#         self.glu_layers = nn.ModuleList([
#             nn.GLU(dim=1)  # Apply GLU along the channel dimension
#             for _ in range(num_time_series)
#         ])
#         self.graph_conv = GraphConvolution(input_dim=2 * hidden_dim, output_dim=hidden_dim)  # Concatenated feature dimension is 2 * hidden_dim

#     def forward(self, time_series_data):
#         conv_outputs = []
#         glu_outputs = []
#         for i, ts_data in enumerate(time_series_data):
#             conv_out = self.conv_layers[i](ts_data)
#             conv_out = F.relu(conv_out)  # Apply ReLU after convolution
#             conv_outputs.append(conv_out)

#             glu_out = self.glu_layers[i](ts_data)
#             glu_outputs.append(glu_out)
#         merged_features = torch.cat([torch.cat(conv_outputs, dim=1), torch.cat(glu_outputs, dim=1)], dim=1)  # Concatenate convolutional and GLU outputs along the channel dimension
#         graph_conv_out = self.graph_conv(merged_features)
#         return graph_conv_out

# # Example usage
# num_time_series = 3
# num_channels = 1
# kernel_size = 3
# hidden_dim = 64
# graph_conv_input_dim = 2 * hidden_dim  # Concatenated feature dimension
# graph_conv_output_dim = 64  # Adjust based on your task
# graph_conv = GraphConvolution(input_dim=graph_conv_input_dim, output_dim=graph_conv_output_dim)

# # Initialize your time series data
# time_series_data = [torch.randn(batch_size, num_channels, sequence_length) for _ in range(num_time_series)]

# # Create and forward through the network
# model = ClusterTS(num_time_series=num_time_series, num_channels=num_channels, kernel_size=kernel_size, hidden_dim=hidden_dim)
# output = model(time_series_data)


C-mean, C/500, cat(C), factor = 0.25, using W, no norm X, cluster=5  
tensor([0.7832, 0.6285, 0.5781, 0.9582, 0.8451, 0.6135, 0.5953, 0.6806, 0.5118,
        0.7671, 0.8329, 1.5210, 1.7983, 1.3789, 1.0746, 1.1554, 0.2643, 0.4089,
        0.4150, 0.6700, 1.1099, 1.3029, 0.5401, 0.6389, 0.2358, 0.2266],

tensor([3.7696e-01, 3.3365e-01, 3.4405e-01, 3.4341e-01, 1.1247e+00, 1.9260e+00,
        3.7696e-01, 4.7058e-01, 1.2311e-01, 1.5041e-01, 1.4726e-01, 1.4382e+00,
        2.4234e+00, 1.6105e+00, 1.5961e-01, 2.5519e-01, 4.4617e-01, 4.5403e-01,
        3.5307e-01, 5.2215e-01, 6.8282e-01, 1.5623e+00, 3.8356e-01, 6.4139e-01,
        1.6950e-03, 1.6287e-03]

In [ ]:
G.get_edge_list()[0].shape

In [ ]:
A = torch.tensor(G.W.toarray()).float() #Using W as a float() tensor
edge_index, _ = dense_to_sparse(A)

In [ ]:
edge_index.shape

In [ ]:
edge_index, _ = dense_to_sparse(torch.tensor(G.A.toarray()))
edge_index